In [1]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

import pathlib
import json
import logging
import time


In [2]:
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('treeresults.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item))
        self.file.write(line)
        return item

In [3]:

class QuotesSpider(scrapy.Spider):
    name = "quotes"
#     start_urls = ['http://quotes.toscrape.com/page/1/']
    start_urls = ['https://aggie-horticulture.tamu.edu/ornamentals/natives/INDEXSCIENTIFIC.HTM']
    custom_settings = {
#         'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEEDS':{ pathlib.Path('treeresult.json' ): {'format': 'json'}}
    }
    
    rules = (Rule(LinkExtractor(allow=(), restrict_xpaths=('//dd',)), callback="parse", follow= True),
    )
    
    def parse(self, response):
#         for quote in response.css('div.quote'):
        for quote in response.xpath('//dd'):    
            yield {'text':quote.xpath('a/text()').extract_first(),
                   'href':quote.xpath('a/@href').extract_first()
#                 'text': quote.css('span.text::text').extract_first(),
#                 'author': quote.css('span small::text').extract_first(),
#                 'tags': quote.css('div.tags a.tag::text').extract(),
  
                   }
            
            

In [4]:
class TreeSpider_aggie(scrapy.Spider):
    name = "quotes"
    allowed_domains= ['aggie-horticulture.tamu.edu']
    start_urls = ['https://aggie-horticulture.tamu.edu/ornamentals/natives/INDEXSCIENTIFIC.HTM']
    base_url = 'https://aggie-horticulture.tamu.edu/'
    
    custom_settings = {
#         'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEEDS':{ pathlib.Path('treeresult.json' ): {'format': 'json'}}
    }

    def parse(self, response):
        links = response.xpath('//dd/a/@href').extract()
        print('there a are ',len(links), ' links')
        for href in links:
            url = response.urljoin(str.upper(href))
            time.sleep(0.5)
            yield scrapy.Request(url, callback = self.parse_item)

    
    def parse_item(self, response):
        table = response.xpath('//table//tr/td')
        entry = table[1:]
        try:
            item = dict()
            item['Common names'] = entry.xpath('font/b/text()').extract_first()
            item['Latin name'] = entry.xpath('i/text()').extract_first()
            item['Family'] = entry.xpath('p/text()').extract_first()[1:]
            yield item
        except:
            print('no entry')
        pass
            

In [5]:
class TreeSpider_lbj(scrapy.Spider):
    name = "quotes"
    allowed_domains= ['www.wildflower.org']
    base_url = 'https://www.wildflower.org/plants/'
    start_urls = ['https://www.wildflower.org/plants/combo.php?start=0&distribution=TX&habit=habit_tree&1&pagecount=10&pagecount=100']
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEEDS':{ pathlib.Path('treeresult_lbj.json' ): {'format': 'json'}}}

    def parse(self, response):
        pages = response.xpath('//a[contains(@href,"start=")]')
        for p in pages[0:4]:
            # print(p.xpath('@href').extract())
            url = self.base_url + p.xpath('@href').extract_first()
            # print(url)
            yield scrapy.Request(url, callback=self.parse_pages)
        yield scrapy.Request(self.start_urls[0], callback=self.parse_pages)
            
    def parse_pages(self, response):
        links = response.xpath('//td/a/@href').extract()
        print('parsing ',len(links), ' pages...')
        for href in links:
            url = response.urljoin(href)
            # print(url)
            time.sleep(0.1)
            yield scrapy.Request(url, callback=self.parse_items)
        return None
        
    def parse_items(self, response):
        table = response.xpath('//h3[@class = "tax"]')
        try:
            item = dict()
            item['Latin name'] = table[0].xpath('i/text()').extract_first()
            item['Common names'] = table[1].xpath('text()').extract_first()
            item['Family'] = table[2].xpath('a/text()').extract_first()+' ' + table[2].xpath('text()').extract_first()
            yield item
        except:
            print('no entry')
        return None
    
            
            

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(TreeSpider_lbj)
process.start()

2020-08-18 16:17:09 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2020-08-18 16:17:09 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.0 (default, Jun 28 2018, 08:04:48) [MSC v.1912 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 3.0, Platform Windows-10-10.0.17134-SP0
2020-08-18 16:17:09 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-08-18 16:17:09 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


https://www.wildflower.org/plants/combo.php?start=100&distribution=TX&habit=habit_tree&1&pagecount=100
https://www.wildflower.org/plants/combo.php?start=200&distribution=TX&habit=habit_tree&1&pagecount=100
https://www.wildflower.org/plants/combo.php?start=300&distribution=TX&habit=habit_tree&1&pagecount=100
https://www.wildflower.org/plants/combo.php?start=100&distribution=TX&habit=habit_tree&1&pagecount=100
there a are  100  links
https://www.wildflower.org/plants/result.php?id_plant=FOAN
https://www.wildflower.org/plants/result.php?id_plant=FRAL3
https://www.wildflower.org/plants/result.php?id_plant=FRAM2
https://www.wildflower.org/plants/result.php?id_plant=FRBE
https://www.wildflower.org/plants/result.php?id_plant=FRBE2
https://www.wildflower.org/plants/result.php?id_plant=FRBEB
https://www.wildflower.org/plants/result.php?id_plant=FRCA13
https://www.wildflower.org/plants/result.php?id_plant=FRCA3
https://www.wildflower.org/plants/result.php?id_plant=FRCU
https://www.wildflower.org

In [7]:
%ls

import pandas as pd

df = pd.read_json('./treeresult_lbj.json')
df.head(100)
df.to_csv('native_trees_lbj.csv')

 Volume in drive C is Windows
 Volume Serial Number is 7486-FF21

 Directory of C:\Users\crm0376\Projects\tree_scraper

08/18/2020  04:20 PM    <DIR>          .
08/18/2020  04:20 PM    <DIR>          ..
08/18/2020  04:20 PM    <DIR>          .ipynb_checkpoints
08/12/2020  12:39 PM            24,243 native_trees.csv
08/12/2020  12:50 PM            25,875 native_trees.xlsx
08/18/2020  04:16 PM            13,569 native_trees_lbj.csv
08/18/2020  04:19 PM            32,484 tree_crawler.ipynb
08/13/2020  12:58 PM            68,528 treeresult.json
08/18/2020  04:20 PM            51,565 treeresult_lbj.json
08/18/2020  04:20 PM            50,903 treeresults.jl
               7 File(s)        267,167 bytes
               3 Dir(s)  104,850,403,328 bytes free
